# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [4]:
import pandas as pd
with open('./I94_SAS_Labels_Descriptions.SAS') as f:
        f_content = f.read()
        f_content = f_content.replace('\t', '')
def code_mapper(file, idx):
        """From udacity knowledge base.
        Sources lookup values from the SAS file. """       
        f_content2 = f_content[f_content.index(idx):]
        f_content2 = f_content2[:f_content2.index(';')].split('\n')
        f_content2 = [i.replace("'", "") for i in f_content2]
        dic = [i.split('=') for i in f_content2[1:]]
        dic = dict([i[0].strip(), i[1].strip()] for i in dic if len(i) == 2)
        print(dic)
        return dic
def creating_df_from_data_dictionary(lookup_column, col_name):
    result = code_mapper(f_content, lookup_column)
    df = pd.DataFrame.from_dict(result, orient='index', columns=[col_name])
    df = df.reset_index()
    df.columns = ['{}_cd'.format(col_name), col_name]
    return df
df = creating_df_from_data_dictionary("I94VISA", "country_birth")  
#df = creating_df_from_data_dictionary("i94prtl", "local_airline")
#df = creating_df_from_data_dictionary("i94addrl", "state")
df

{'1': 'Business', '2': 'Pleasure', '3': 'Student'}


,country_birth_cd,country_birth
0,1,Business
1,2,Pleasure
2,3,Student


In [7]:
# Do all imports and installs here
import pandas as pd
import re

def regex_lev(x):
    code= x.split('-')[1]
    return code 
airport = pd.read_csv('airport-codes_csv.csv')
airport.head()
m = airport[airport['iso_country'] == "US"][['iso_country','iso_region','local_code','iata_code']]
m['iso_region'] = m['iso_region'].apply(lambda x: regex_lev(x))
#l = m.groupby(['iso_country','iso_region']).count()
m[ ~m['iata_code'].isnull()].shape
m

In [2]:
import pandas as pd

In [35]:
airport = pd.read_csv('us-cities-demographics.csv',delimiter=';')
airport.columns
#airport['Race'].value_counts()
m = airport[['City', 'State', 'Median Age', 'Male Population', 'Female Population','Total Population', 'Number of Veterans', 'Foreign-born', 'Average Household Size', 'State Code']]
m.drop_duplicates().shape
airport[['City','State Code']].drop_duplicates().shape
#airport[airport['City']=='Columbus']
#
#airport[(airport['City']=='Abilene') & (airport['State']=='Texas')]

(596, 2)

In [9]:
immigration = pd.read_csv('immigration_data_sample.csv')
immigration.head()
immigration.columns

Index(['Unnamed: 0', 'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port',
       'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
       'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd',
       'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum',
       'airline', 'admnum', 'fltno', 'visatype'],
      dtype='object')

In [23]:
immigration['i94port'].value_counts()
m = immigration.loc[immigration['i94port'] =='X96']
m.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
271,2163396,4348794.0,2016.0,4.0,525.0,525.0,X96,20567.0,2.0,VQ,...,NaN,M,1968.0,10222016,F,NaN,VES,9.435467e+10,MANAD,B2


In [8]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df = pd.read_csv(fname)
print(df.columns)
print(df.head)

Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
       'Country', 'Latitude', 'Longitude'],
      dtype='object')
<bound method NDFrame.head of                  dt  AverageTemperature  AverageTemperatureUncertainty  \
0        1743-11-01               6.068                          1.737   
1        1743-12-01                 NaN                            NaN   
2        1744-01-01                 NaN                            NaN   
3        1744-02-01                 NaN                            NaN   
4        1744-03-01                 NaN                            NaN   
5        1744-04-01               5.788                          3.624   
6        1744-05-01              10.644                          1.283   
7        1744-06-01              14.051                          1.347   
8        1744-07-01              16.082                          1.396   
9        1744-08-01                 NaN                            NaN   
10       1744-

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [1]:
# Read in the data here
import pandas as pd
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")
print(df.shape)
#print(df[['depdate','arrdate']].value_counts())

(3096313, 28)


AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [2]:
print(df['cicid'].drop_duplicates().shape)
#print(df['depdate'].value_counts())
#print(df['arrdate'].value_counts())
#print(df.columns)

(3096313,)


In [8]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.